In [1]:
import torch
flag = torch.cuda.is_available()
print(flag)

if flag:
    print(torch.cuda.get_device_name(0))
    print(torch.rand(3,3).cuda()) 

True
NVIDIA RTX A5000
tensor([[0.6601, 0.0063, 0.1479],
        [0.9316, 0.0662, 0.5008],
        [0.3951, 0.6088, 0.6714]], device='cuda:0')


In [1]:
import sys
import os

# 确定项目根目录（假设当前工作目录是项目的根目录）
project_root = os.path.abspath("..")
sys.path.insert(0, project_root)

import pickle
from ai import ai_battle, MinimaxAI, MCTSAI
from game.chess_game import ChessGame
from CelestialVault.instances import ExampleThreadManager


class TrainDataThread(ExampleThreadManager):
    def get_args(self, obj: object):
        return (mcts_ai_0, mcts_ai_0, ChessGame((5, 5), 2), False)
    
    def process_result(self):
        all_training_data = []
        result_dict = self.get_result_dict()
        for d in result_dict:
            over_game = result_dict[d]
            history_board = over_game.history_board
            history_move = over_game.history_move
            for step in history_board:
                if step+1 not in history_board.keys():
                    continue
                board = self.process_board(history_board, step)
                if (board, history_move[step+1]) in all_training_data:
                    continue
                all_training_data.append((board, history_move[step+1]))
        return all_training_data
    
    def process_board(self, history_board, step):
        color = 1 if step % 2 == 0 else -1
        
        processed_board = []
        for row in history_board[step]:
            processed_row = []
            for cell in row:
                processed_cell = cell + [color]
                if processed_cell[0] == float("inf"):
                    processed_cell[0] = 5
                processed_row.append(processed_cell)
            processed_board.append(processed_row)
        return processed_board



minimax_ai = MinimaxAI(5)
mcts_ai_0 = MCTSAI(10000, flag=True)
mcts_ai_1 = MCTSAI(10000, flag=False)

train_data_threader = TrainDataThread(
            ai_battle,
            thread_num=50,
            tqdm_desc='trainDataProcess',
            show_progress=True)


In [27]:

from time import strftime, localtime

def load_data(file_path):
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    return data

def save_data(data):
    data_size = len(data)
    now_time = strftime("%m-%d-%H", localtime())
    pickle.dump(data, open(f"train_data/all_training_data({now_time})({data_size}).pkl", "wb"))

def train_data(train_num):
    train_data_threader.start(range(train_num), "serial")
    train_data_threader.handle_error()
    all_training_data = train_data_threader.process_result()

    save_data(all_training_data)
    

    return all_training_data

# all_training_data = []
# for _ in range(1):
#     all_training_data += train_data(100)

# len(all_training_data)

In [69]:
save_data(all_training_data)

In [85]:
all_training_data = load_data(r"G:\Project\Celestial-Chess\ai\train_data\all_training_data(06-13-23)(1506).pkl")

In [67]:
old_data = all_training_data[:]
all_training_data = []

for i in old_data:
    if i in all_training_data:
        continue
    all_training_data.append(i)

len(all_training_data)

9598

In [86]:
import torch
from torch.utils.data import DataLoader, Dataset

class ChessDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        board_state, move = self.data[idx]
        board_state = torch.tensor(board_state, dtype=torch.float32)
        move = move[0] * 5 + move[1]
        return board_state, move

dataset = ChessDataset(all_training_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

len(dataset)

1506

In [87]:
from time import strftime, localtime
import torch.nn as nn
import torch.optim as optim
from ai.deeplearning import ChessModel, DeepLearningAI

# 设置CuDNN选项
torch.backends.cudnn.benchmark = True

model = ChessModel().cuda() # 初始化模型，并将其移动到GPU上
criterion = nn.CrossEntropyLoss() # 定义交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001) # 定义Adam优化器

# 训练循环
num_epochs = 10  # 训练10个epoch
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        # 调整输入的维度，并将其移动到GPU上
        # inputs 的原始形状是 (batch_size, height, width, channels)，也就是 (32, 5, 5, 3)
        # inputs.permute(0, 3, 1, 2) 会将 inputs 的维度从 (32, 5, 5, 3) 转换为 (32, 3, 5, 5)
        inputs = inputs.permute(0, 3, 1, 2).cuda()  # (batch_size, channels, height, width)
        labels = labels.cuda()

        # 清零梯度
        optimizer.zero_grad()

        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()

        # 累积损失
        running_loss += loss.item()
        if i % 100 == 99:  # 每100个batch打印一次loss
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')
data_size = len(dataset)
now_time = strftime("%m-%d-%H", localtime())
torch.save(model.state_dict(), f'models/chess_ai_model({now_time})({data_size}).pth')


Finished Training


In [108]:
from ai.deeplearning import DeepLearningAI

# 加载深度学习AI
deep_learning_ai_0 = DeepLearningAI('models/chess_ai_model(06-15-17)(1471).pth')
deep_learning_ai_1 = DeepLearningAI('models/chess_ai_model(06-15-17)(936).pth')
deep_learning_ai_2 = DeepLearningAI('models/chess_ai_model(06-15-17)(15475).pth')
deep_learning_ai_3 = DeepLearningAI('models/chess_ai_model(06-15-17)(9598).pth')

# 测试AI
test_minimax = MinimaxAI(6)
test_mcts_0 = MCTSAI(1000, flag=True)
test_mcts_1 = MCTSAI(10000, flag=True)

# 与其他AI算法进行对战
# ai_battle(deep_learning_ai, minimax_ai, display=True)
ai_battle(test_minimax, deep_learning_ai_0, ChessGame((5, 5), 2), display=True)



游戏开始！
蓝方AI: DeepLearningAI
红方AI: MinimaxAI

tensor([[[ -9.4477,   1.8316,   3.1183,   2.7062,  -5.7609],
         [ -7.2529,   2.9624,   0.7789,   3.4943,  -5.6922],
         [ -5.0785,   1.5514,   1.4713,   2.7061,  -3.8614],
         [ -7.8593,   1.8678,   0.1315,   1.5162,  -4.3138],
         [-12.3476,  -7.2359,  -8.7611,  -6.9403, -13.3612]]], device='cuda:0')
[
  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
  [[0, 0], [0, 0], [1, 1], [2, 2], [1, 1]],
  [[0, 0], [0, 0], [0, 0], [1, 1], [0, 0]],
  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
]
第1步: 蓝方 落子在 (1, 3)
获胜概率: 0.00%
分数: 2.5
用时: 0.01s

[
  [[ 0,  0], [ 0,  0], [-1,  1], [-2,  2], [-1,  1]],
  [[ 0,  0], [ 0,  0], [ 1,  1], [ 1,  3], [ 1,  1]],
  [[ 0,  0], [ 0,  0], [ 0,  0], [ 1,  1], [ 0,  0]],
  [[ 0,  0], [ 0,  0], [ 0,  0], [ 0,  0], [ 0,  0]],
  [[ 0,  0], [ 0,  0], [ 0,  0], [ 0,  0], [ 0,  0]]
]
第2步: 红方 落子在 (0, 3)
获胜概率: 0.00%
分数: -2.5
用时: 38.51s

tensor([[[-3.3562,  3.4032,    -in